In [4]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install --upgrade selenium

Note: you may need to restart the kernel to use updated packages.


In [36]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
import json
import string
import os
import time

# Path to your webdriver
path_to_webdriver = "C:/Users/prave/OneDrive/Desktop/Current_Projects/My_try1/msedgedriver.exe"

# Create a new instance of the Edge driver
s = Service(executable_path=path_to_webdriver)
driver = webdriver.Edge(service=s)


# Function to sanitize file names
def sanitize_file_name(file_name):
    return "".join(c for c in file_name if c.isalnum() or c in (' ',)).rstrip()

# Iterate over each letter
for letter in string.ascii_lowercase:
    #     element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, f'[id="075c9eb4-1ba8-407c-9bb0-3a223d0b0314js-{letter}"].js-enabled')))
    #     element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//a[contains(@class, "js-enabled") and text()="{letter.upper()}"]')))
    #     element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, f'a.js-enabled:contains("{letter.upper()}")')))
    driver.get("https://my.clevelandclinic.org/health/treatments")

    # EC.element_to_be_clickable
    # EC.visibility_of_element_located
    # Click on BROWSER A-Z
    browser_az = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "js-library-search-nav__browse-btn")))
    browser_az.click()
    time.sleep(2)
    try:
        element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//a[contains(@class, "js-enabled") and text()="{letter.upper()}"]')))    
        element.click()
        time.sleep(2)
    except:
        continue
    # Get the page source and create a BeautifulSoup object
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract the links you want
    links = soup.find_all('a', class_="index-list-link")

    # Create a directory for the letter if it doesn't exist
    os.makedirs(letter, exist_ok=True)

    # Iterate over each link
    for link in links:
        # Get the URL and the name of the link
        url = link['href']
        name = sanitize_file_name(link.text)

        # Send a GET request to the URL
        response = requests.get(url)

        # If the GET request is successful, the status code will be 200
        if response.status_code == 200:
            # Get the content of the response
            page_content = response.content

            # Create a BeautifulSoup object and specify the parser
            soup = BeautifulSoup(page_content, 'html.parser')

            # Extract all the text from the webpage
            data = soup.get_text()

            # Check if data is not empty
            if data.strip():
                # Save the data to a JSON file in the directory
                with open(f"{letter}/{name}.json", 'w',  encoding='utf-8') as f:
                    json.dump(data, f, indent=4)
    url = None
    name = None
    response = None
    page_content = None
    soup = None
    data = None    
        

# Close the driver
driver.quit()